# 1. NVFLARE

## SLURM 指令函式化

In [15]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
     1090376_[6-9]      ct2k Neuropix p00acy00 PD       0:00     36 (Resources) 
           1090839     ct224 oplsaa_C r0752403 PD       0:00      4 (Priority) 
           1090837     ct224 oplsaa_C r0752403 PD       0:00      4 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions) 
           1090838     ct224 oplsaa_C r0752403 PD       0:00      4 (Priority) 
           1090841     ct224 CCCl_qui sharon47 PD       0:00      2 (Priority) 
           1090840      ct56 CCCl_qui sharon47 PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions) 
           1090814      ct56 Atest.in tommykuo PD       0:00      1 (Priority) 
           1090815      ct56     A.in tommykuo PD       0:00      1 (Priority) 
           1090816      ct56     B.in tommykuo PD       0:00      1 (Priority) 
           1090863      ct5

In [16]:
# https://man.twcc.ai/@TWCC-III-manual/ryyo0tsuu
def SLURM(cmd):
    ## SLURM 內容, 請修改 ---> Email
    SLURM='''#!/work/c00cjz002/binary/bash5.0/bin/bash
#SBATCH -A MST109178        # 計畫代號
#SBATCH -J CPU_T3_C34       # 工作代號 (標籤, 可自行定義)
#SBATCH -p ngs12G           # 工作區塊 
#SBATCH -c 3                # 使用的CPU核心數
#SBATCH --mem=12g           # 使用的記憶體容量 
#SBATCH --mail-user=summerhill001@gmail.com    # 請修改為您的信向
#SBATCH --mail-type=BEGIN,END                  # 指定送出email時機 可為NONE, BEGIN, END, FAIL, REQUEUE, ALL
#SBATCH -o log/%j.logi      # 執行記錄檔案儲存於log目錄底下
#SBATCh -w cpn3671
'''
    myCmd = SLURM + cmd
    
    ## 儲存上述內容 SLURM.sh
    import time
    slurm_shell = 'slurm/'+time.strftime("%Y-%m-%d_%H-%M-%S")+'.sh'
    
    f = open(slurm_shell, "w")
    f.write(myCmd)
    f.close()    

    ## 執行SLURM
    #!sbatch SLURM.sh
    jobID=(subprocess.check_output('sbatch '+slurm_shell+' |awk \'{print $4}\'', shell=True,text=True))
    return jobID

## 建立目錄
import subprocess
!mkdir -p slurm
!mkdir -p log

## nvflare Server

In [17]:
## nvflare Server
cmd='''
## -> 啟動 admin
unset http_proxy
unset https_proxy
cd /home/c00cjz00/nvflare_data/jupyter
source condaInit.sh use_env nvflare 3.8
bash server.sh /home/c00cjz00/abc
pid=$!
sleep 1

# -> 等待nvflar結束才會離線
wait $pid
'''

## 送出工作到計算節點電腦
jobID = SLURM(cmd)
print(jobID.strip())

1090909


In [18]:
import time
#jobID
## 觀看執行狀況
hit='none'
while hit!='cpn':
    tmp=!squeue -u `whoami` | sed -e 's/^[[:space:]]*//' |grep $jobID
    x = tmp[0].split()
    adminHostname=x[7]
    hit=adminHostname[:3]
    if  hit!='cpn':
        time.sleep( 5 )

adminHostname

'cpn3883'

In [19]:
## 觀看執行狀況
!squeue -u `whoami` | grep "$jobID\|JOBID" 

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
           1090909    ngs12G CPU_T3_C c00cjz00  R       0:04      1 cpn3883 


In [20]:
# https://man.twcc.ai/@TWCC-III-manual/ryyo0tsuu
def SLURM(cmd):
    ## SLURM 內容, 請修改 ---> Email
    SLURM='''#!/work/c00cjz002/binary/bash5.0/bin/bash
#SBATCH -A MST109178        # 計畫代號
#SBATCH -J GPU_T3_C34       # 工作代號 (標籤, 可自行定義)
#SBATCH -p ngs1gpu         # 工作區塊 
#SBATCH -c 6                # 使用的CPU核心數
#SBATCH --mem=90g           # 使用的記憶體容量 
#SBATCH --gres=gpu:1        # 使用的GPU數 請參考Queue資源設定
#SBATCH --mail-user=summerhill001@gmail.com    # 請修改為您的信向
#SBATCH --mail-type=BEGIN,END                  # 指定送出email時機 可為NONE, BEGIN, END, FAIL, REQUEUE, ALL
#SBATCH -o log/%j.logi      # 執行記錄檔案儲存於log目錄底下
#SBATCh -w cpn3671
'''
    myCmd = SLURM + cmd
    
    ## 儲存上述內容 SLURM.sh
    import time
    slurm_shell = 'slurm/'+time.strftime("%Y-%m-%d_%H-%M-%S")+'.sh'
    
    f = open(slurm_shell, "w")
    f.write(myCmd)
    f.close()    

    ## 執行SLURM
    #!sbatch SLURM.sh
    jobID=(subprocess.check_output('sbatch '+slurm_shell+' |awk \'{print $4}\'', shell=True,text=True))
    return jobID

## 建立目錄
import subprocess
!mkdir -p slurm
!mkdir -p log

## nvflare site-1

In [21]:
## nvflare site-1
cmd='''
## -> 啟動 site-1
unset http_proxy
unset https_proxy
server='''+adminHostname+'''
cd /home/c00cjz00/nvflare_data/jupyter
source condaInit.sh use_env nvflare 3.8
bash site-1.sh ${server} site-1 /home/c00cjz00/abc &
pid=$!
sleep 1

# -> 等待nvflar結束才會離線
wait $pid
'''

## 送出工作到計算節點電腦
jobID = SLURM(cmd)
print(jobID.strip())

1090910


## nvflare site-2

In [22]:
## nvflare site-2
cmd='''
## -> 啟動 site-2
unset http_proxy
unset https_proxy
server='''+adminHostname+'''
cd /home/c00cjz00/nvflare_data/jupyter
source condaInit.sh use_env nvflare 3.8
bash site-1.sh ${server} site-2 /home/c00cjz00/abc &
pid=$!
sleep 1

# -> 等待nvflar結束才會離線
wait $pid
'''

## 送出工作到計算節點電腦
jobID = SLURM(cmd)
print(jobID.strip())

1090911


## 計算節點電腦

In [23]:
## 觀看執行狀況
!squeue -u `whoami` | grep "$jobID\|JOBID" 

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
           1090910   ngs1gpu GPU_T3_C c00cjz00  R       0:00      1 gpn3001 
           1090911   ngs1gpu GPU_T3_C c00cjz00  R       0:00      1 gpn3001 
           1090909    ngs12G CPU_T3_C c00cjz00  R       0:09      1 cpn3883 


In [13]:
## 刪除JOB 
!scancel 1090905   1090906  1090903                 

In [24]:
## nvflare admin
cmd='''
## -> 啟動 admin
server='''+adminHostname+'''
cd /home/c00cjz00/nvflare_data/jupyter
source condaInit.sh use_env nvflare 3.8
bash admin.sh ${server} /home/c00cjz00/abc
'''

## 送出工作到計算節點電腦
print(cmd)


## -> 啟動 admin
server=cpn3883
cd /home/c00cjz00/nvflare_data/jupyter
source condaInit.sh use_env nvflare 3.8
bash admin.sh ${server} /home/c00cjz00/abc



## 自家電腦